In [1]:
import os 
import pandas as pd 
import numpy as np 
import cv2

In [2]:
data = pd.read_csv('/Users/HPhuc/Practice/12. classification/vinbigdata/output/csv/data_train.csv', index_col=0)
data 

,image_ids,class_name,Aortic enlargement,Atelectasis,Calcification,Cardiomegaly,Consolidation,ILD,Infiltration,Lung Opacity,Nodule Mass,Other lesion,Pleural effusion,Pleural thickening,Pneumothorax,Pulmonary fibrosis,image_paths
0,a259009c4eeacf9ccf4f8db4bf3e7714,['No finding' 'No finding' 'No finding'],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/Users/HPhuc/Practice/12. classification/vinbi...
1,ac2a615b3861212f9a2ada6acd077fd9,['Aortic enlargement' 'Pleural thickening' 'Pl...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,/Users/HPhuc/Practice/12. classification/vinbi...
2,211fb0b1fae0691e21159bcff77672a7,['No finding' 'No finding' 'No finding'],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/Users/HPhuc/Practice/12. classification/vinbi...
3,003cfe5ce5c0ec5163138eb3b740e328,['No finding' 'No finding' 'No finding'],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/Users/HPhuc/Practice/12. classification/vinbi...
4,3fb2fa3cceb68338ecf497cb3a2a318d,['No finding' 'No finding' 'No finding'],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/Users/HPhuc/Practice/12. classification/vinbi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0fa3a43de5d5204d9be82a1969c1b6c8,['No finding' 'No finding' 'No finding'],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/Users/HPhuc/Practice/12. classification/vinbi...
96,d69c970d9c790b8813f5b0d0964e6a55,['No finding' 'No finding' 'No finding'],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/Users/HPhuc/Practice/12. classification/vinbi...
97,a0af7d64eecad23a41730cef8dbf2f11,['No finding' 'No finding' 'No finding'],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/Users/HPhuc/Practice/12. classification/vinbi...
98,fb40ca51c74b4be05cc855629211b406,['No finding' 'No finding' 'No finding'],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/Users/HPhuc/Practice/12. classification/vinbi...


In [3]:
import os 
import cv2
from keras.models import load_model

IMG_W = 1024
IMG_H = 1024
IMG_C = 3

unet_path = '/Users/HPhuc/Practice/12. classification/vinbigdata/output/best_model.h5'
img_path = '/Users/HPhuc/Practice/12. classification/vinbigdata/data/train/0e130f7333967146f0d7bbd94a8ae047.png'

In [4]:
# import multiprocessing
# multiprocessing.cpu_count()

In [5]:
from repos.utils import multiprocess

def get_imgs(idc):
  row = data.iloc[idc]
  img_path = row['image_paths']
  id_ = row['image_ids']
  img = cv2.imread(img_path)

  return img, id_

ds = multiprocess(get_imgs, range(0, len(data)), workers=4)

...multiprocessing...


100%|██████████| 100/100 [00:00<00:00, 110.72it/s]


In [13]:
img, id_ = get_imgs(0)
img = np.array(img[np.newaxis])
print(img.shape)
unet = load_model(unet_path, compile=False)
msk = unet.predict(img)



(1, 1024, 1024, 3)


In [17]:
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

dloader = DataLoader(ds, batch_size=64, num_workers=6)

In [18]:
from tqdm import tqdm 
from repos.utils import get_progress

unet = load_model(unet_path, compile=False)

def predict_msk(dloader):
  final_msks = []
  final_ids  = []
  final_imgs = []

  for imgs, ids in tqdm(dloader): 
    imgs = np.array(imgs)
    msks = unet.predict(imgs)
    msks = (msks*255).astype(np.uint8)
    msks = (msks>127).astype(int)
     
    final_imgs.append(imgs)
    final_msks.append(msks)
    final_ids.append(ids)
  
  final_imgs = np.concatenate(final_imgs)
  final_msks = np.concatenate(final_msks)
  final_ids  = np.concatenate(final_ids)
  
  return final_imgs, final_msks, final_ids

final_imgs, final_msks, final_ids = predict_msk(dloader)
print(final_imgs.shape)
print(final_msks.shape)
print(final_ids.shape)

100%|██████████| 2/2 [02:46<00:00, 83.49s/it] 


(100, 1024, 1024, 3)
(100, 1024, 1024, 1)
(100,)


In [20]:
def pad_image(img, axes, crop_size):
    shapes = np.array(img.shape)
    axes = np.array(axes)
    sizes = np.array(shapes[axes])
    diffs = sizes - np.array(crop_size)
    for diff, axis in zip(diffs, axes):
        left = abs(diff) // 2
        right = (left + 1) if diff % 2 != 0 else left
        if diff >= 0: 
          continue
        elif diff < 0:
          img = np.pad(img, [(left, right) if i == axis else (0, 0) for i in range(len(shapes))])
    return img
    

In [26]:
from scipy import ndimage
import skimage
from matplotlib.patches import Rectangle
from skimage.measure import label
import matplotlib.pyplot as plt

train_path = '/Users/HPhuc/Practice/12. classification/vinbigdata/output/output_xray/'

def Crop_LCP(idc, size, return_path):
    # Toạ độ của largest connected component
    msk = final_msks[idc]
    img = final_imgs[idc]
    id_ = final_ids[idc]
    slice_y, slice_x = ndimage.find_objects(msk.squeeze(), 1)[0]

    x_min = slice_x.start 
    x_max = slice_x.stop
    y_min = slice_y.start
    y_max = slice_y.stop

    xy = (slice_x.start, slice_y.start)
    h, w = slice_y.stop - slice_y.start, slice_x.stop - slice_x.start
    nw, nh = int(w/0.875), int(h/0.875)
    dw, dh = (nw-w)//2, (nh-h)//2

    top   = max(slice_y.start-dh, 0)
    left  = max(slice_x.start-dw, 0)
    bot   = min(slice_y.stop+dh, 1024)
    right = min(slice_x.stop+dw, 1024)

    img_crop = img[top : bot, left : right]
    
    """ Check whether img size > = 256, if not -> padding
    """
    if np.any(np.array(img_crop.shape[:2]) < size): 
      img_crop = pad_image(img=img_crop, axes=(0,1), crop_size=[size,size])
      print('padding:', img_crop.shape)

    assert np.all(np.array(img_crop.shape[:2]) >= size), f'{id_}' + ' crop size' + f'{img_crop.shape[:2]}' ' must be larger than' + f'{size, size}'

    if return_path: np.save(f'{train_path}' + id_ + '.png', img_crop)

    if idc % 100 == 0:
      fig, ax1 = plt.subplots(1, 1, figsize=(12,8))
      ax1.imshow(img_crop, cmap=plt.cm.gray)
      plt.show()

    print(img_crop.shape)

    # return img_crop 


In [28]:
crop_lung = multiprocess(Crop_LCP, range(len(data)), size=256, return_path=True)


...multiprocessing...


  0%|          | 0/100 [00:00<?, ?it/s]

(632, 737, 3)
(611, 792, 3)
(633, 833, 3)
(499, 848, 3)
(584, 929, 3)
(811, 995, 3)
(720, 817, 3)
(619, 881, 3)
(673, 1009, 3)
(589, 820, 3)
(677, 946, 3)
(762, 921, 3)


  1%|          | 1/100 [00:00<01:15,  1.32it/s]

(694, 768, 3)
(738, 882, 3)


...

 93%|█████████▎| 93/100 [00:04<00:00, 21.68it/s]

(636, 981, 3)
(634, 876, 3)
(614, 774, 3)
(678, 773, 3)

 97%|█████████▋| 97/100 [00:04<00:00, 22.34it/s]


(690, 873, 3)


100%|██████████| 100/100 [00:05<00:00, 19.97it/s]


In [29]:
import os 

len(os.listdir(train_path))

100